In [1]:
#coding: utf-8
import warnings
warnings.filterwarnings("ignore")
import os
import time
import random
import jieba
import sklearn
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn import metrics
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from numpy import vstack

def MakeWordsSet(words_file):
    """
    函数说明:读取文件里的内容，并去重
    Parameters:
        words_file - 文件路径
    Returns:
        words_set - 读取的内容的set集合
    """
    words_set = set()
    with open(words_file, 'r') as fp:
        for line in fp.readlines():
            word = line.strip()#.decode("utf-8")
            if len(word)>0 and word not in words_set: # 去重
                words_set.add(word)
    return words_set

def TextProcessing(folder_path, test_size=0.2):
########################################### 使用 #######################################################
# folder_path = './Database/SogouC/Sample'
    """
    函数说明:中文文本处理
    Parameters:
        folder_path - 文本存放的路径
        test_size - 测试集占比，默认占所有数据集的百分之20
    Returns:
        all_words_list - 按词频降序排序的训练集列表
        train_data_list - 训练集列表
        test_data_list - 测试集列表
        train_class_list - 训练集标签列表
        test_class_list - 测试集标签列表
    """
    folder_list = os.listdir(folder_path)
    data_list = []
    class_list = []
###################################################################################
# data_list  ： 全部文本信息list
# class_lsit ： 每个txt文件的类别标签
    # 类间循环
    for folder in folder_list:
        new_folder_path = os.path.join(folder_path, folder)
        files = os.listdir(new_folder_path)
        # 类内循环
        j = 1
        for file in files:
            if j > 100: # 每类text样本数最多100
                break
            with open(os.path.join(new_folder_path, file), 'r') as fp:
               raw = fp.read()
           
            word_cut = jieba.cut(raw, cut_all=False) # 精确模式，返回的结构是一个可迭代的genertor
            word_list = list(word_cut) # genertor转化为list，每个词unicode格式
          
            data_list.append(word_list)
            class_list.append(folder)#.decode('utf-8')
            j += 1

    # 划分训练集和测试集
    data_class_list = list(zip(data_list, class_list))
    random.shuffle(data_class_list)
    index = int(len(data_class_list)*test_size)#+1
    train_list = data_class_list[index:]
    test_list = data_class_list[:index]
    train_data_list, train_class_list = zip(*train_list)
    test_data_list, test_class_list = zip(*test_list)

    # 统计词频放入all_words_dict
    all_words_dict = {} 
    for word_list in train_data_list:
        for word in word_list:
            #if all_words_dict.has_key(word):
            if word in all_words_dict.keys():
                all_words_dict[word] += 1
            else:
                all_words_dict[word] = 1

    # 根据键的值倒序排序
    all_words_tuple_list = sorted(all_words_dict.items(), key=lambda f:f[1], reverse=True) # 内建函数sorted参数需为list
    #all_words_list = list(zip(*all_words_tuple_list))[0]
    all_words_list = list(zip(*all_words_tuple_list))[0]

    return all_words_list, train_data_list, test_data_list, train_class_list, test_class_list

def words_dict(all_words_list, deleteN, stopwords_set=set()):
    """
    函数说明:文本特征选取
    Parameters:
        all_words_list - 训练集所有文本列表
        deleteN - 删除词频最高的deleteN个词
        stopwords_set - 指定的结束语
    Returns:
        feature_words - 特征集
    """
    # 选取特征词
    feature_words = []
    n = 1
    for t in range(deleteN, len(all_words_list), 1):
        if n > 1000: # feature_words的维度1000
            break
        # 如果这个词不是数字，并且不是指定的结束语，并且单词长度大于1小于5，那么这个词就可以作为feature_word
        if not all_words_list[t].isdigit() and all_words_list[t] not in stopwords_set and 1<len(all_words_list[t])<5:
            feature_words.append(all_words_list[t])
            n += 1
    return feature_words

def TextFeatures(train_data_list, test_data_list, feature_words):
    """
    函数说明:根据feature_words将文本向量化
    Parameters:
        train_data_list - 训练集
        test_data_list - 测试集
        feature_words - 特征集
    Returns:
        train_feature_list - 训练集向量化列表
        test_feature_list - 测试集向量化列表
    """
    def text_features(text, feature_words):
        text_words = set(text)
        features = [1 if word in text_words else 0 for word in feature_words]
        return features
    train_feature_list = [text_features(text, feature_words) for text in train_data_list]
    test_feature_list = [text_features(text, feature_words) for text in test_data_list]
    return train_feature_list, test_feature_list

def TextClassifier(train_feature_list, test_feature_list, 
                    train_class_list, test_class_list,):
    """
    函数说明:分类器
    Parameters:
        train_feature_list - 训练集向量化的特征文本
        test_feature_list - 测试集向量化的特征文本
        train_class_list - 训练集分类标签
        test_class_list - 测试集分类标签
        classifier      - 使用的分类器函数
        params          - 分类器要优化的参数
    Returns:
        test_accuracy - 分类器精度
    """
    train_feature_list = list(train_feature_list)
    train_class_list = list(train_class_list)
    test_feature_list = list(test_feature_list)
    test_class_list = list(test_class_list)

    X_train = train_feature_list
    Y_train  = train_class_list

    X_train_c = np.copy(train_feature_list)
    Y_train_c  = np.copy(train_class_list)
    
    X_val  = test_feature_list
    Y_val = test_class_list

    len_X_train = len(X_train)
    len_X_val = len(X_val)

    X = vstack([X_train,X_val])
    X = np.array(X)
    Y_train.extend(Y_val)
    Y = np.array(Y_train)

    #Mark the training-validation splits
    train_i = np.ones((len_X_train,), dtype = int) * -1
    valid_i = np.zeros((len_X_val,), dtype = int)
    split_fold = np.concatenate((train_i, valid_i))
    ps = PredefinedSplit(split_fold)
    
    params = {'alpha':np.linspace(0.0001,1,10000)}
    classifier = MultinomialNB()
    
    param_search = GridSearchCV(classifier,
                            params, 
                    scoring=metrics.make_scorer(metrics.f1_score, average='macro'),
                                cv=ps,
                                return_train_score=True)
    param_search.fit(X,Y)
    results = param_search.cv_results_
    best_params = param_search.best_params_ 
    
    clf = MultinomialNB(alpha = best_params['alpha'])
    clf.fit(X_train_c,Y_train_c)
    Y_pred = clf.predict(X_val)
    test_accuracy = metrics.f1_score(Y_val, Y_pred, average='macro')

    return test_accuracy

if __name__ == '__main__':
    acc = []
    # 文本预处理
    folder_path = './Database/SogouC/Sample'
    all_words_list, train_data_list, test_data_list, train_class_list, test_class_list = TextProcessing(folder_path, test_size=0.2)

    # 生成stopwords_set
    stopwords_file = './stopwords_cn.txt'
    stopwords_set = MakeWordsSet(stopwords_file)
    '''
    # 文本特征提取和分类
    deleteN = 450
    feature_words = words_dict(all_words_list, deleteN, stopwords_set)
    
    #acc = []
    #for i in range(10):
    train_feature_list, test_feature_list = TextFeatures(train_data_list, test_data_list, feature_words)

    m = TextClassifier(train_feature_list, test_feature_list, 
                    train_class_list, test_class_list)
    acc.extend([m])
    print(acc)'''
        '''
        vectorizer = CountVectorizer()  # 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j],表示j词在i类文本下的词频
        transformer = TfidfTransformer()    # 该类会统计每个词语的tf-idf权值
        # 第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
        tfidf = transformer.fit_transform(vectorizer.fit_transform(all_comment_list))
        word = vectorizer.get_feature_names()   # 获取词袋模型中的所有词语
        weight = tfidf.toarray()    # 将tf-idf矩阵抽取出来，元素weight[i][j]表示j词在i类文本中的tf-idf权重,4w条评论需要4GB内存
        ##########'''
        

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.629 seconds.
Prefix dict has been built succesfully.


In [2]:
train_data_list

(['\u3000',
  '\u3000',
  '\u3000',
  '本报',
  '北京',
  '５',
  '月',
  '９',
  '日讯',
  ' ',
  '记者',
  '潘跃',
  '今天',
  '从',
  '中国红十字会总会',
  '举办',
  '的',
  '“',
  '博爱',
  '论坛',
  '”',
  '上',
  '获悉',
  '：',
  '目前',
  '，',
  '全国',
  '已有',
  '６',
  '４',
  '１',
  '个',
  '县',
  '（',
  '市',
  '、',
  '区',
  '）',
  '开展',
  '试点工作',
  '，',
  '有',
  '１',
  '．',
  '６',
  '３',
  '亿',
  '农民',
  '参加',
  '了',
  '合作医疗',
  '。',
  '试点',
  '地区',
  '农民',
  '的',
  '医疗',
  '负担',
  '有所',
  '减轻',
  '，',
  '因',
  '病',
  '致贫',
  '、',
  '因',
  '病',
  '返贫',
  '的',
  '情况',
  '有所',
  '缓解',
  '。',
  '\n',
  '\u3000',
  '\u3000',
  '据',
  '有关',
  '资料',
  '显示',
  '，',
  '我国',
  '医疗',
  '资源分配',
  '极',
  '不',
  '均衡',
  '，',
  '占',
  '全国',
  '人口',
  '２',
  '０',
  '％',
  '的',
  '城市',
  '人口',
  '占有',
  '我国',
  '卫生',
  '资源',
  '的',
  '８',
  '０',
  '％',
  '，',
  '而',
  '占',
  '全国',
  '人口',
  '８',
  '０',
  '％',
  '的',
  '农村',
  '人口',
  '仅',
  '占有',
  '２',
  '０',
  '％',
  '的',
  '卫生',
  '资源',
  '。',
  '近年来',
  '，',
  '由于',
  '医疗',
  

In [3]:
[" ".join(x) for x in train_data_list ]

['\u3000 \u3000 \u3000 本报 北京 ５ 月 ９ 日讯   记者 潘跃 今天 从 中国红十字会总会 举办 的 “ 博爱 论坛 ” 上 获悉 ： 目前 ， 全国 已有 ６ ４ １ 个 县 （ 市 、 区 ） 开展 试点工作 ， 有 １ ． ６ ３ 亿 农民 参加 了 合作医疗 。 试点 地区 农民 的 医疗 负担 有所 减轻 ， 因 病 致贫 、 因 病 返贫 的 情况 有所 缓解 。 \n \u3000 \u3000 据 有关 资料 显示 ， 我国 医疗 资源分配 极 不 均衡 ， 占 全国 人口 ２ ０ ％ 的 城市 人口 占有 我国 卫生 资源 的 ８ ０ ％ ， 而 占 全国 人口 ８ ０ ％ 的 农村 人口 仅 占有 ２ ０ ％ 的 卫生 资源 。 近年来 ， 由于 医疗 服务 保障 的 城乡差异 ， 大城市 的 人均 寿命 比 农村 高 １ ２ 年 ， 而 贫困地区 儿童 死亡率 则 是 大城市 的 ９ 倍 。',
 "\n \n \n \n 《 大长 今 》 调查 \n \n 1 、 你 看 《 大长 今 》 吗 ？ \n \n 不看 \n \n 赶上 就 看 \n \n 正在 追着 看 \n \n 看过 n 遍     \n \n 2 、 《 大长 今 》 吸引 你 的 是 \n \n 情节 曲折 ， 悬念 迭起 \n \n 俊男靓女 美食 \n \n 爱情 亲情 友情 感人 ， 台词 经典 \n \n 励志 向上 ， 善 和 美 打动 人心     \n \n 3 、 你 认为 日韩剧 成功 在 哪里 \n \n 细节 取胜 ， 平淡 中见 真情 \n \n 情节 设计 合理 ， 戏剧性 强 \n \n 人物 立体 丰满 ， 贴近生活 ， 比较 真实 \n \n 演员 表演 分寸 适当 ， 不 夸张 \n \n 包含 很多 时尚 元素 ， 画面 唯美     \n \n           \n var   NewWin   =   null ;   function   WinOpen ( url )   { if ( ! NewWin   | |   NewWin . closed )   { NewWin = LoadWin ( url , ' win _ poll ' , 418 , 300 ) ; } else { Ne